In [687]:
import numpy as np
import matplotlib.pyplot as plt

In [688]:
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [689]:
df = pd.read_json("/media/influx/Programming/Projects/project-final-random/Recommender/users.json")

In [690]:
df_train = df.drop(['dateOfBirth', 'name', 'email', 'profile_pic', 'connectionId'], axis=1)

In [691]:
df_train

,id,universityLocation,field,workplace,specialties
0,1,Tehran,AA,Tesla,"[Database, Front-End, Crypto]"
1,2,Princeton,BB,Apple,"[Full-Stack, Machine Learning, Fast Type]"
2,3,Cambridge,AA,Meta,"[Blockchain, C++, AI]"
3,4,Cambridge,BB,Intel,"[Photoshop, Game, Django]"
4,5,Princeton,DD,AMD,"[Full-Stack, Machine Learning, Photoshop, Game..."
...,...,...,...,...,...
1997,1998,Carnegie,DD,AMD,"[PHP, Machine Learning, Python, Database, Data..."
1998,1999,Cambridge,AA,AMD,"[Machine Learning, Data, Hardware, Django]"
1999,2000,Stanford,CC,Digikala,"[Photoshop, Machine Learning, C++]"
2000,2001,Stanford,AA,Cisco,[Photoshop]


In [692]:
df_train = pd.get_dummies(df_train, columns=["field"])
df_train = pd.get_dummies(df_train, columns=["universityLocation"])
df_train = pd.get_dummies(df_train, columns=["workplace"])

In [693]:
df_train

,id,specialties,field_AA,field_BB,field_CC,field_DD,universityLocation_Amirkabir,universityLocation_Berkeley,universityLocation_Cambridge,universityLocation_Carnegie,...,workplace_Cisco,workplace_Digikala,workplace_Google,workplace_Intel,workplace_Meta,workplace_Microsoft,workplace_NVIDIA,workplace_Snapp,workplace_TesUberla,workplace_Tesla
0,1,"[Database, Front-End, Crypto]",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,"[Full-Stack, Machine Learning, Fast Type]",0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,"[Blockchain, C++, AI]",1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,4,"[Photoshop, Game, Django]",0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
4,5,"[Full-Stack, Machine Learning, Photoshop, Game...",0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,1998,"[PHP, Machine Learning, Python, Database, Data...",0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1998,1999,"[Machine Learning, Data, Hardware, Django]",1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1999,2000,"[Photoshop, Machine Learning, C++]",0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2000,2001,[Photoshop],1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [694]:
spec = set(s for user in df_train['specialties'] for s in user)
for special in spec:
    df_train[special] = [0 for i in range(2002)]

In [695]:
i = 0
for user in df_train['specialties']:
    for s in user:
        df_train.at[i, s] = 1
    i+=1
    
df_train = df_train.drop(['specialties', 'id'], axis=1)

In [696]:
df_train

,field_AA,field_BB,field_CC,field_DD,universityLocation_Amirkabir,universityLocation_Berkeley,universityLocation_Cambridge,universityLocation_Carnegie,universityLocation_ETH,universityLocation_Esfahan,...,Python,PHP,Front-End,IT,C#,Machine Learning,Verilog,Crypto,Java,Full-Stack
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,0,0,0,1,0,0,0,1,0,0,...,1,1,0,0,0,1,0,0,0,0
1998,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1999,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2000,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [697]:
df_train.columns

Index(['field_AA', 'field_BB', 'field_CC', 'field_DD',
       'universityLocation_Amirkabir', 'universityLocation_Berkeley',
       'universityLocation_Cambridge', 'universityLocation_Carnegie',
       'universityLocation_ETH', 'universityLocation_Esfahan',
       'universityLocation_Harvard', 'universityLocation_MIT',
       'universityLocation_Oxford', 'universityLocation_Princeton',
       'universityLocation_Sharif', 'universityLocation_Stanford',
       'universityLocation_Tehran', 'workplace_AMD', 'workplace_Adobe',
       'workplace_Amazon', 'workplace_Apple', 'workplace_Bazaar',
       'workplace_Cisco', 'workplace_Digikala', 'workplace_Google',
       'workplace_Intel', 'workplace_Meta', 'workplace_Microsoft',
       'workplace_NVIDIA', 'workplace_Snapp', 'workplace_TesUberla',
       'workplace_Tesla', 'Django', 'Algorithm', 'Fast Type', 'Hardware',
       'Game', 'Software', 'Data', 'Database', 'Back-End', 'Photoshop',
       'Blockchain', 'AI', 'C++', 'Python', 'PHP', 'Fron

In [698]:
ability_power = list()
for user in df_train.values:
    if(user[0] == 1):
        power = 4
    elif user[1] == 1:
        power = 3
    elif user[2] == 1:
        power = 2
    elif user[3] == 1:
        power = 1
    
    ability = 0
    for i in range(33, 55):
        if(user[i] == 1):
            ability += 1
            
    ability_power.append(np.power(ability/(np.exp(2*power)), 2))
    

In [699]:
df_train['ability_power'] = ability_power

In [700]:
df_train = df_train.drop(['field_AA', 'field_BB', 'field_CC', 'field_DD', 'Algorithm', 'Database', 'Game'], axis=1)

In [701]:
df_train = df_train.drop(['Python', 'C#', 'Software', 'Back-End', 'Fast Type', 'Verilog'], axis=1)

In [702]:
df_train = df_train.drop(['Photoshop', 'AI', 'Java', 'PHP', 'Data', 'Front-End', 'Hardware', 'Django'], axis=1)

In [703]:
df_train = df_train.drop(['C++', 'Crypto', 'Full-Stack', 'IT', 'Machine Learning', 'Blockchain'], axis=1)

In [704]:
df_train

,universityLocation_Amirkabir,universityLocation_Berkeley,universityLocation_Cambridge,universityLocation_Carnegie,universityLocation_ETH,universityLocation_Esfahan,universityLocation_Harvard,universityLocation_MIT,universityLocation_Oxford,universityLocation_Princeton,...,workplace_Digikala,workplace_Google,workplace_Intel,workplace_Meta,workplace_Microsoft,workplace_NVIDIA,workplace_Snapp,workplace_TesUberla,workplace_Tesla,ability_power
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1.401885e-07
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,7.654035e-06
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1.401885e-07
3,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,7.057847e-06
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2.527063e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.620911e-02
1998,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.401885e-07
1999,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,4.178962e-04
2000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.125352e-07


In [705]:
dsc = df_train['ability_power'].values
sc = StandardScaler()
dsc = dsc.reshape(-1,1)
dsc = sc.fit_transform(dsc)

In [706]:
dsc

array([[-0.6050116 ],
       [-0.60431829],
       [-0.6050116 ],
       ...,
       [-0.56646494],
       [-0.60501415],
       [ 2.25989016]])

In [707]:
# df_train["ability_power"] = dsc

In [708]:
df_train

,universityLocation_Amirkabir,universityLocation_Berkeley,universityLocation_Cambridge,universityLocation_Carnegie,universityLocation_ETH,universityLocation_Esfahan,universityLocation_Harvard,universityLocation_MIT,universityLocation_Oxford,universityLocation_Princeton,...,workplace_Digikala,workplace_Google,workplace_Intel,workplace_Meta,workplace_Microsoft,workplace_NVIDIA,workplace_Snapp,workplace_TesUberla,workplace_Tesla,ability_power
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1.401885e-07
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,7.654035e-06
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1.401885e-07
3,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,7.057847e-06
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2.527063e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.620911e-02
1998,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.401885e-07
1999,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,4.178962e-04
2000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.125352e-07


In [677]:
X = df_train.values

In [678]:
def estimate_gaussian(X): 
    m, n = X.shape
    
    mu = np.mean(X, axis=0)
    var = np.var(X, axis=0)
        
    return mu, var

In [679]:
def ProbabilityAnomaly(X):
    mu , var = estimate_gaussian(X)
    
    P_X = dict()
    idx = 0
    for user in X:
        p_x_i = (1/np.sqrt(2*np.pi*var))*np.exp(-(np.power(user-mu, 2))/(2*var))
        P_X[idx] = np.prod(p_x_i)
        idx += 1
    
    return P_X

In [680]:
print((ProbabilityAnomaly(X)))

{0: 12.088593316974965, 1: 2.144287812567621, 2: 4.001011560143596, 3: 0.32789646532382016, 4: 2.8993483056121274, 5: 0.256145400400396, 6: 0.1771070965034289, 7: 0.0739259366024411, 8: 6.203745479762765, 9: 1.7315377374100351, 10: 20.662328217772846, 11: 0.3302777613541595, 12: 10.250411863176083, 13: 0.5896857415040924, 14: 2.1068293839166117, 15: 0.6839506204116033, 16: 6.736656396845903, 17: 5.6089302820201965, 18: 6.8294518131912945, 19: 0.023011454374645902, 20: 2.2267692820074, 21: 28.58604535048014, 22: 2.6676166093310267, 23: 0.08339154615723228, 24: 0.16277639538998293, 25: 1.9668682178184687, 26: 0.13675363538955126, 27: 0.16124738023564886, 28: 20.67281549000671, 29: 10.7141381069127, 30: 0.5460340071329305, 31: 0.20722819651660315, 32: 4.636327017463576, 33: 0.7623959589499004, 34: 5.687335120614876, 35: 15.073311103226713, 36: 0.5743574322994717, 37: 20.826047182139455, 38: 0.7020159179094246, 39: 0.5084110441197066, 40: 10.116250163612849, 41: 0.18851795946868685, 42: 6.

In [681]:
x = ProbabilityAnomaly(X)

In [682]:
x = dict(sorted(x.items(), key=lambda item: item[1]))

In [683]:
x[2000]

0.04070709399711736

In [684]:
x

{2001: 3.712636713303222e-09,
 97: 0.00108250975097715,
 805: 0.0015692302630332016,
 1317: 0.0017690181656877522,
 311: 0.001806501710638018,
 423: 0.002130647508265261,
 1048: 0.002655142065240867,
 312: 0.0030105650037997905,
 402: 0.0031542470567756742,
 314: 0.0033811142041122125,
 776: 0.0034705258918283767,
 1365: 0.0037082496362560867,
 1557: 0.003781101322093579,
 405: 0.004339317378294687,
 212: 0.004515703909918089,
 1991: 0.004515703909918089,
 116: 0.005090624642983508,
 494: 0.00652901199497569,
 564: 0.00652901199497569,
 1042: 0.006777677944722981,
 1085: 0.006777677944722981,
 1345: 0.0074126587349805,
 1572: 0.0074126587349805,
 1052: 0.007736626886776258,
 1764: 0.007736626886776258,
 1088: 0.008721132562768553,
 424: 0.009729681482102582,
 253: 0.009831471067278372,
 473: 0.009831471067278372,
 165: 0.0108807065643579,
 257: 0.011390304690227176,
 460: 0.011390304690227176,
 1233: 0.011533482157858219,
 1797: 0.011672735975103111,
 1639: 0.01192442324931817,
 1081: 